In [4]:
import pandas as pd
from loader import *
import matplotlib.pyplot as plt
import jieba
import jieba.analyse



In [ ]:
doc, q, cho, ans = json_parser()

In [ ]:
doc[0]

In [ ]:
frag_len = [ len(frag) for frag_doc in doc for frag in frag_doc]
max(frag_len), min(frag_len)

In [ ]:
plt.title('Fragment len')
plt.hist(frag_len)
plt.show()

In [ ]:
allfragment_len = [sum([len(e) for e in ele]) for ele in doc]
plt.hist(allfragment_len)
plt.show()


In [1]:
def cat_sents(idx, end):
    indices = [idx]
    head = tail = idx
    dir = 0
    while len(indices) != 16:
        print(head, tail)
        if tail+1< end and dir==0: 
            tail+=1
            indices.append(tail)
        elif head-1>=0 and dir==1:
            head-=1   
            indices.append(head)
        dir=not dir
    return indices

In [ ]:
cat_sents(48, 50)

In [11]:
import difflib

In [12]:
difflib.SequenceMatcher(None, '123', '56161').quick_ratio()

0.25

In [ ]:
import numpy as np
a = np.array([
    [3, 40, 20,],
    [1, 56, 21],
    [441, 560, 28],
    [11, 5, 2],
])


In [ ]:
ind = np.argpartition(a, -3, axis=0)[-3:]
ind


In [ ]:
ind[:, 0]

In [ ]:
ind[np.argsort(a[ind])]


In [ ]:
import torch 

In [ ]:
a = torch.rand((1,2,3))
b = torch.rand((1,3,3))
a,b

In [ ]:
torch.cat((a, b), dim=1)

In [1]:
with open('./data/stop_word.txt') as f:
    text = f.read()
    text = text.split('\n')

In [2]:
[1,2] + [2,3]

[1, 2, 2, 3]

In [13]:
tags = jieba.analyse.extract_tags('醫師：我們6月30號抽血，免疫細胞指數還不錯這樣子，然後病毒量也是OK，阿血糖，糖化血色素7.2，還不錯，所以控制的還不錯，所以表示這段時間身體狀況應該還好。民眾：因為我有重訓和訓練……醫師：低密度膽固醇101是正常，是可以的，然後因為有一點點尿酸稍微高一點點，但是還不需要吃藥，就是飲食稍微注意一下就好，會造成尿酸高一點的食物稍微控制一下就好。民眾：我不知道哪些。醫師：不')
tags

In [26]:
import spacy
from spacy.lang.zh import Chinese
from spacy.language import Language

text = '醫師：我們6月30號抽血，免疫細胞指數還不錯這樣子，然後病毒量也是OK，阿血糖，糖化血色素7.2，還不錯，所以控制的還不錯，所以表示這段時間身體狀況應該還好。民眾：因為我有重訓和訓練……醫師：低密度膽固醇101是正常，是可以的，然後因為有一點點尿酸稍微高一點點，但是還不需要吃藥，就是飲食稍微注意一下就好，會造成尿酸高一點的食物稍微控制一下就好。民眾：我不知道哪些。醫師：不。'

In [33]:
@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    
    position = ['民眾', '個管師', '醫師', '女醫師', '護理師', '家屬', '藥師', ]
    position = [ role+'：' for role in position] + \
                [ role + ele + '：' for role in position for ele in ['A', 'B', '1', '2']]

    for token in doc[:-1]:
#         print(token, token.i)
        if token.text in position:
            doc[token.i].is_sent_start = True
        elif token.i>0 and doc[token.i-1].text in ['，', '？', '。', ]:
            doc[token.i].is_sent_start = True
        else:
            doc[token.i].is_sent_start = False
    return doc
nlp = spacy.load("zh_core_web_sm")
nlp = Chinese()
nlp = Chinese.from_config({"nlp": {"tokenizer":{"segmenter": "pkuseg"}}})
nlp.tokenizer.initialize(pkuseg_model="mixed", pkuseg_user_dict='./data/pkuseg_user_dict.txt')
nlp.add_pipe("sentencizer")
nlp.add_pipe("set_custom_boundaries")
doc = nlp(text)


In [28]:
[sen.text for sen in doc.sents]

['醫師：我們6月30號抽血，',
 '免疫細胞指數還不錯這樣子，',
 '然後病毒量也是OK，',
 '阿血糖，',
 '糖化血色素7.2，',
 '還不錯，',
 '所以控制的還不錯，',
 '所以表示這段時間身體狀況應該還好。',
 '民眾：因為我有重訓和訓練……',
 '醫師：低密度膽固醇101是正常，',
 '是可以的，',
 '然後因為有一點點尿酸稍微高一點點，',
 '但是還不需要吃藥，',
 '就是飲食稍微注意一下就好，',
 '會造成尿酸高一點的食物稍微控制一下就好。',
 '民眾：我不知道哪些。',
 '醫師：不。']

In [ ]:
a = torch.tensor([
    [
        [1,2,3],
        [0,0,0],
        [3,4,3]
    ],
    [
        [1,2,3],
        [1,2,3],
        [3,4,3]
    ],
], dtype=torch.float)
b = torch.tensor([
    [
        [1,2,3],
        [3,4,3]
    ],
    [
        [1,2,3],
        [3,4,3]
    ],
], dtype=torch.float)
cosine_sim()(a, b)

In [ ]:
cosine_sim()(a, b)

In [ ]:
torch.max(cosine_sim()(a, b), 1)

In [ ]:
# output = torch.randn(2, 4, 768)
# choi_output = torch.randn(2, 3, 768)
cosine_sim(None)(a, b)

In [ ]:
vec1=a
vec2=b

vec2 = torch.transpose(vec2, 1, 2)
norm1 = torch.linalg.norm(vec1, dim=2)
norm2 = torch.linalg.norm(vec2, dim=1)   
norm1 = norm1.unsqueeze(-1).expand(2, vec1.size(1), vec1.size(2))
norm2 = norm2.unsqueeze(2).expand(2, vec2.size(2), vec2.size(1))
norm2 = torch.transpose(norm2, 1, 2)
eledot = torch.matmul(torch.div(vec1, norm1), torch.div(vec2, norm2))

# print(torch.mul(norm1.transpose(0, 1), norm2))
# cos_sim = torch.div(eledot ,torch.matmul(norm1.transpose(0, 1), norm2))
eledot

In [ ]:
norm2 = norm2.unsqueeze(2).expand(2, vec2.size(2), vec2.size(1))
norm2 = torch.transpose(norm2, 1, 2)

In [ ]:
' rock '

In [ ]:
from loader import QA_Dataset
from transformers import AutoTokenizer
config = 'hfl/chinese-xlnet-base'
tokenizer = AutoTokenizer.from_pretrained(config)


In [ ]:
example = QA_Dataset(tokenizer, './data/train_fold1.json').__getitem__(1)
example['input_ids'].shape 

In [ ]:
tokenizer.decode(example['input_ids'][0][0])

In [ ]:
tokenizer.decode(ins['q_input_ids'][0])

In [ ]:
import pandas as pd
from loader import *
import matplotlib.pyplot as plt
from transformers import AutoTokenizer


config = 'hfl/chinese-xlnet-base'
tokenizer = AutoTokenizer.from_pretrained(config)
doc, q, cho, ans = json_parser()

In [ ]:
import jieba
jieba.load_userdict('./data/dict.txt') 
# lijieba.lcut(text))

In [ ]:
def find_entity(strs, ent, dir='f'):
    try:
        idx = strs.index(ent)
        idx = idx-1 if dir=='f' else idx+1
        return strs[idx]
    except:
        return 0
def query_propress(query):
    valid_ent = ['民眾', '醫師', '個管師', '家屬']
    rules=[ 
        {'p':'關於','dir':'b'}, {'p':'建議','dir':'f'},
        {'p':'是', 'dir':'b'}, {'p':'不是', 'dir':'b'},
        {'p':'請問', 'dir':'b'}
    ]
    processed_q = []
    for q in query:
        tokens = jieba.lcut(q)
        # print(tokens)
        res = ''
        for ele in rules:
            res = find_entity(tokens, ele['p'], dir=ele['dir'])  

            if res in valid_ent: break
        # Clearing invalid ent
        if not res in valid_ent:
            res= tokens[0] if tokens[0] in valid_ent else ''
            
        processed_q.append(res)
    return processed_q


In [ ]:
propressed_query = query_propress(q)
list(zip(q, propressed_query))

In [ ]:
pipe =  doc_preprocessing(config)
d = pipe(doc, q)

# ori_len = [len(window) for frag in d for window in frag['input_ids']]
# plt.hist(ori_len, color = 'black', cumulative = False)
# plt.show()

In [ ]:
tokenizer.decode(d[0]['q_input_ids'][0])

In [ ]:
plt.hist(strQ2B(ans))
plt.show()


## LOADER

In [ ]:
from torch.utils.data import DataLoader
from loader import QA_Dataset
from torch.nn.utils.rnn import pad_sequence
import torch

config = 'hfl/chinese-xlnet-base'
lr = 1e-3

In [ ]:
import  torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
a = torch.ones(25, 300)
b = torch.ones(22, 300)
c = torch.ones(15, 300)
pad_sequence([a, b, c], batch_first=True,).shape

In [ ]:
def fragment_pad(*dcts):
    combined_dict = {}
    for k in set(dcts[0]):
        if not k=='label' and not k=='choice':
            print(k, [d[k].shape for d in dcts])
            combined_dict.update({
                k:pad_sequence([d[k].squeeze(0) for d in dcts], batch_first=True)
            })
        if k=='label':
             combined_dict.update({
                'label':torch.stack([d['label'] for d in dcts]) 
             })
    return combined_dict

def collate_fn(batch):
    batched_dict = fragment_pad(*batch)
    return batched_dict

train_ds = QA_Dataset(config, './data/Train_qa_ans.json')
train_loader = DataLoader(
    train_ds, 
    batch_size=4, 
    num_workers=1, 
    shuffle=True, 
    collate_fn=collate_fn
)
train_loader.__iter__().__next__()
# model = QA_Model(config, lr=lr)

In [ ]:
from loader import QA_Dataset

In [ ]:
from transformers import XLNetTokenizer, XLNetModel
import torch

config = 'hfl/chinese-xlnet-base'
tokenizer = XLNetTokenizer.from_pretrained(config)
tokenizer.add_special_tokens({'additional_special_tokens':['<q>']})
model = XLNetModel.from_pretrained(config)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
tokenizer([
    ('預計的性行為', '預計的性行為'),
    ('預預預預預預', '預預預預預預'),

])['input_ids'][0]

In [ ]:
tokenizer.decode(19)

In [ ]:
c ='預計的性行為，前面兩個小時吃兩顆，24小時後，再吃一顆，再一個24小時之後吃一顆'

ids = tokenizer([c, c[0:30]], max_length=40, padding='max_length').input_ids
tokenizer.decode(ids[0])

In [ ]:
for ele in ids[0]:
    print(    tokenizer.decode(ele))


In [ ]:
tokenizer.decode(ids[1])

## Model

In [ ]:
h = model(**encoding).last_hidden_state

In [ ]:
from transformers.modeling_utils import SequenceSummary


In [ ]:
s = SequenceSummary(config)
getattr(config, 'summary_type', 'last')

In [ ]:
import torch

a = torch.randn(2,3,7)
a


In [ ]:
torch.cat([a,a,a], dim=2).shape

In [ ]:
index = torch.tensor([
    [ 1,2,1 ],
    [ 1,2,2 ],
])

max_val = [a.index_select(1, ele[1])[ele[0]] for ele in enumerate(index)]
# a.index_select(1, index)
max_val

In [ ]:
 torch.cat(max_val).view(2,-1, 7)

In [ ]:
# MEM 

In [ ]:
from transformers import AutoModel, AutoConfig

model_name = 'hfl/chinese-xlnet-base'

config = AutoConfig.from_pretrained(
        model_name,
        n_layer=3,
)
query_encoder = AutoModel.from_pretrained(
        model_name,
        config=config,
)



In [ ]:
q

In [ ]:
from torch.nn import CrossEntropyLoss
import torch

In [ ]:
l = CrossEntropyLoss()
y = torch.empty(1, dtype=torch.long).random_(3)
pred = torch.tensor([[-0.2, -0.3, 0.2]], requires_grad=True)
pred, y

In [ ]:
l(pred, y)